<a href="https://colab.research.google.com/github/Lexiiz3417/sentiment-analysis/blob/main/sentimen%20model" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Persiapan (Install Library & Download Data NLTK)
Ini langkah awal buat install dan unduh semua yang dibutuhkan.

In [8]:
!pip install pandas nltk scikit-learn tensorflow joblib
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import joblib
import nltk

# Unduh data stopwords Indonesia dari NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 2. Muat & Bersihkan Data
Blok ini berfungsi buat ngambil data dari CSV dan ngebersihin teksnya.
[Download datasetnya disini](https://www.kaggle.com/datasets/achmadshidiq/corpus-bahasa-indonesia-label/data)

In [9]:
data = pd.read_csv('Corpus Bahasa Indonesia (Label).csv')

# Perbaikan untuk mengatasi KeyError: 'teks'
data.rename(columns={'kalimat': 'teks', 'sentiment': 'sentiment'}, inplace=True)

# Ganti nilai sentiment 2 menjadi 0 (Negatif/Netral)
# Jika ingin 3 kelas, jangan jalankan baris ini dan pastikan label y-nya 0, 1, 2
# data['sentiment'] = data['sentiment'].replace(2, 0)

list_stopwords = stopwords.words('indonesian')
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in list_stopwords]
    return ' '.join(filtered_tokens)

data['teks_bersih'] = data['teks'].apply(preprocess_text)

In [10]:
# Cek nilai unik di kolom sentiment
print("\nNilai unik di kolom 'sentiment':")
print(data['sentiment'].unique())

# Cek tipe data dari kolom sentiment
print("\nTipe data kolom 'sentiment':")
print(data['sentiment'].dtype)


Nilai unik di kolom 'sentiment':
[0 1 2]

Tipe data kolom 'sentiment':
int64


# 3. Ubah Teks ke Angka (TF-IDF)
Di sini kita ubah teks yang udah bersih jadi matriks angka.

In [11]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data['teks_bersih'])

print("\nShape dari matriks TF-IDF:")
print(vectors.shape)


Shape dari matriks TF-IDF:
(12609, 13304)


In [12]:
# Kode untuk menyimpan vectorizer
!pip install joblib
import joblib
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')

['tfidf_vectorizer.joblib']

# 4. Bagi Data & One-Hot Encoding
Blok ini memisahkan data menjadi dua bagian, untuk diajarin dan dites.

In [13]:
from tensorflow.keras.utils import to_categorical

# Ambil kolom sentiment sebagai label (y)
X = vectors
y = data['sentiment']

# Ubah label (0, 1, 2) menjadi one-hot encoding
y_one_hot = to_categorical(y)

# Bagi data menjadi 80% untuk latih dan 20% untuk uji
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

print("\nData berhasil dibagi dan diubah menjadi One-Hot Encoding!")
print(f"Jumlah data latih: {X_train.shape[0]}")
print(f"Jumlah data uji: {X_test.shape[0]}")
print(f"Bentuk label y_train: {y_train.shape}")


Data berhasil dibagi dan diubah menjadi One-Hot Encoding!
Jumlah data latih: 10087
Jumlah data uji: 2522
Bentuk label y_train: (10087, 3)


# 5. Bangun Model JST Multi-Kelas
Ini bagian inti di mana kita buat arsitektur JST-nya.

In [14]:
# Bangun Model JST Sederhana
model = Sequential()

# Lapisan pertama
model.add(Dense(16, input_shape=(X_train.shape[1],), activation='relu'))

# Lapisan output (diganti jadi 3 neuron dan softmax)
# 3 = jumlah kelas
# 'softmax' = fungsi aktivasi untuk multi-kelas
model.add(Dense(3, activation='softmax'))

# Tampilkan ringkasan model
model.summary()

# Kompilasi Model
# 'categorical_crossentropy' = fungsi loss untuk multi-kelas
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │       212,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,931 (831.76 KB)

 Trainable params: 212,931 (831.76 KB)

 Non-trainable params: 0 (0.00 B)

# 6. Latih Model
Blok terakhir ini buat 'ngajarin' model JST-mu pakai data latih.

In [15]:
# Latih model
# 'epochs' adalah berapa kali model belajar dari seluruh data
# 'batch_size' adalah jumlah sampel per langkah pelatihan
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.4997 - loss: 1.0492 - val_accuracy: 0.5036 - val_loss: 0.9643
Epoch 2/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.5835 - loss: 0.8553 - val_accuracy: 0.6360 - val_loss: 0.8315
Epoch 3/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8135 - loss: 0.6011 - val_accuracy: 0.6614 - val_loss: 0.7633
Epoch 4/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.8885 - loss: 0.4320 - val_accuracy: 0.6681 - val_loss: 0.7507
Epoch 5/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9192 - loss: 0.3246 - val_accuracy: 0.6733 - val_loss: 0.7649
Epoch 6/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9324 - loss: 0.2602 - val_accuracy: 0.6745 - val_loss: 0.7914
Epoch 7/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9381 - loss: 0.2151 - val_accuracy: 0.6733 - val_loss: 0.8230
Epoch 8/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9540 - loss: 0.1752 - val_accur

# 7.  Simpan Model AI

In [16]:
# Simpan model ke dalam file
model.save('sentimen_model.h5')

# Modelmu sekarang tersimpan dengan nama 'sentimen_model.h5' di Colab!
# Kamu bisa download file ini dari menu di samping kiri Colab